In [1]:
#importing libraries

import emoji

In [4]:
#checking the classes

print(type(emoji.emojize(":angry_face:")))
print(type(emoji.UNICODE_EMOJI))

<class 'str'>
<class 'dict'>


In [5]:
#printing some emojis

print(emoji.emojize(":angry_face:"))
print(emoji.emojize(":OK_hand_light_skin_tone:"))

😠
👌🏻


In [6]:
#importing pandas library

import pandas as pd

In [7]:
#reading csv file

data = pd.read_csv("train_emoji.csv",header=None)

In [8]:
#getting first 10 rows

data.head(n=10)

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN
5,I love you mum,0,NaN,NaN
6,Stop saying bullshit,3,NaN,NaN
7,congratulations on your acceptance,2,NaN,NaN
8,The assignment is too long,3,NaN,NaN
9,I want to go play,1,NaN,[3]


In [9]:
#creating our dictionary

emoji_dict = {
    0:":beating_heart:",
    1:":baseball:",
    2:":beaming_face_with_smiling_eyes:",
    3:":disappointed_face:",
    4:":fork_and_knife:"    
}

In [10]:
#printing the emojis from our emoji dictionary

for ix in emoji_dict.values():
    print(emoji.emojize(ix))

💓
⚾
😁
😞
🍴


In [11]:
#assigning the 0th column to 'X_train' variable

X_train = data[0]

#assigning the 1st column to 'Y_train' variable
Y_train = data[1]

In [12]:
#printing the shapes

print(X_train.shape)
print(Y_train.shape)

(132,)
(132,)


In [15]:
#printing the values
print(X_train[0])
print(Y_train[0])

never talk to me again
3


In [16]:
#printing value with corresponding emoji

print(X_train[0])
print(emoji.emojize(emoji_dict[Y_train[0]]))

never talk to me again
😞


In [18]:
#using Standford Glove Training Model

def read_glove_vecs(glove_file):
    with open(glove_file, 'r',encoding='utf8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [22]:
#including the file

words_to_index, index_to_words, word_to_vec_map = read_glove_vecs("glove.6B.50d.txt")

In [25]:
#checking the shape of any random word (say "happy")

word_to_vec_map["happy"].shape

(50,)

In [26]:
#importing numpy library

import numpy as np

In [27]:
#calculating the average vector for a sentence

def avg_vector(words):
    
    ans = word_to_vec_map[words[0]]
   # print(ans)
    for i in range(1,len(words)):
        ans = np.add(ans,word_to_vec_map[words[i]])
    ans = ans/float(len(words))
    return ans

In [30]:
#printing the average vector from "X_train" variable

ans = avg_vector(X_train[0])
print(ans)

[ 9.718460e-02  1.743740e-02  5.284300e-02 -4.368060e-01  2.756132e-01
 -1.426688e-01 -6.120520e-01  4.716140e-01 -6.059740e-01  1.566594e-01
 -8.662400e-02  4.906620e-01 -6.480320e-01 -1.612960e-01  8.897200e-01
  4.867520e-01  4.601260e-02 -8.363342e-02 -2.288840e-01 -4.151500e-01
 -1.140000e-02  6.407600e-01  5.954280e-01  1.780868e-01  7.128668e-01
 -2.072760e+00 -3.161338e-01  1.869920e-01  6.583040e-01 -6.969300e-01
  3.172380e+00  5.411100e-01 -5.941440e-01 -3.028740e-01 -2.285386e-01
 -2.899120e-01  2.006040e-01  1.053064e-01 -3.933000e-02 -4.483020e-01
 -1.353596e-01 -2.031200e-03 -2.454976e-01  1.305460e-01 -1.976000e-02
  2.025340e-02 -1.199070e-01 -2.522034e-01 -2.197784e-01  3.076060e-01]


In [31]:
#similarly creating the average vector for all the rows of "X_train" variable

X_numbers = np.zeros((132,50))

for i in range(X_train.shape[0]):
    X_numbers[i] = avg_vector(X_train[i])

print(X_numbers.shape)
print(X_numbers)

(132, 50)
[[ 0.0971846   0.0174374   0.052843   ... -0.2522034  -0.2197784
   0.307606  ]
 [ 0.00563117  0.59711333 -0.13939717 ... -0.06701233 -0.025773
   0.32180167]
 [ 0.27343125  0.33763875 -0.33053875 ... -0.335001    0.06524875
  -0.04257963]
 ...
 [ 0.29220562  0.1533622  -0.015246   ...  0.032272    0.1313342
   0.472114  ]
 [-0.00701397  0.54196333 -0.19225433 ... -0.80106333 -0.20057967
   0.78087   ]
 [-0.1149985   0.6452325  -0.39727    ...  0.142175   -0.2057708
   0.5231    ]]


In [34]:
#importing keras library

from keras.models import Sequential
from keras.layers import Dense,Input,Activation
from keras.utils import to_categorical

In [35]:
model = Sequential()

In [36]:
#building neural network

model.add(Dense(10, activation='relu', input_shape=(50,)))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [40]:
#printing the shape of "Y_train" variable

print(Y_train.shape)

#defining the onehot means setting that correct emoji to 1 from the list of our emoji dictionary

Y_onehot = to_categorical(Y_train)

#printing the shape of "Y_onehot" variable

print(Y_onehot.shape)

(132,)
(132, 5)


In [41]:
#printing the values of "Y_train" and "Y_onehot" variable

print(Y_train[0])
print(Y_onehot[0])

3
[0. 0. 0. 1. 0.]


In [42]:
model.fit(X_numbers,Y_onehot,epochs=100)

Epoch 1/100
132/132 [==============================] - 0s 2ms/step - loss: 1.8442 - acc: 0.1515
Epoch 2/100
132/132 [==============================] - 0s 83us/step - loss: 1.7950 - acc: 0.1591
Epoch 3/100
132/132 [==============================] - 0s 91us/step - loss: 1.7585 - acc: 0.1667
Epoch 4/100
132/132 [==============================] - 0s 60us/step - loss: 1.7259 - acc: 0.2045
Epoch 5/100
132/132 [==============================] - 0s 76us/step - loss: 1.7008 - acc: 0.2121
Epoch 6/100
132/132 [==============================] - 0s 76us/step - loss: 1.6762 - acc: 0.2197
Epoch 7/100
132/132 [==============================] - 0s 83us/step - loss: 1.6556 - acc: 0.2348
Epoch 8/100
132/132 [==============================] - 0s 83us/step - loss: 1.6373 - acc: 0.2348
Epoch 9/100
132/132 [==============================] - 0s 68us/step - loss: 1.6218 - acc: 0.2424
Epoch 10/100
132/132 [==============================] - 0s 91us/step - loss: 1.6051 - acc: 0.2803
Epoch 11/100
132/132 [========

132/132 [==============================] - 0s 68us/step - loss: 0.8388 - acc: 0.7727
Epoch 85/100
132/132 [==============================] - 0s 60us/step - loss: 0.8322 - acc: 0.7727
Epoch 86/100
132/132 [==============================] - 0s 53us/step - loss: 0.8244 - acc: 0.7803
Epoch 87/100
132/132 [==============================] - 0s 83us/step - loss: 0.8178 - acc: 0.7803
Epoch 88/100
132/132 [==============================] - 0s 83us/step - loss: 0.8112 - acc: 0.8106
Epoch 89/100
132/132 [==============================] - 0s 76us/step - loss: 0.8056 - acc: 0.8106
Epoch 90/100
132/132 [==============================] - 0s 68us/step - loss: 0.7982 - acc: 0.8030
Epoch 91/100
132/132 [==============================] - 0s 60us/step - loss: 0.7915 - acc: 0.8030
Epoch 92/100
132/132 [==============================] - 0s 60us/step - loss: 0.7849 - acc: 0.8030
Epoch 93/100
132/132 [==============================] - 0s 68us/step - loss: 0.7784 - acc: 0.7955
Epoch 94/100
132/132 [===========

In [61]:
#testing

#providing any random texts(in lower case)
text = "i want to play baseball"

avg_v = avg_vector(text.split())
print(avg_v.shape)
avg_v= avg_v.reshape(1,50)
print(avg_v.shape)

(50,)
(1, 50)


In [62]:
#predicting the suitable emoji label for the given text

emoji_label_predict = np.argmax(model.predict(avg_v))

In [63]:
#printing the predicted emoji by its label

print(emoji.emojize(emoji_dict[emoji_label_predict]))

⚾
